In [1]:
import pandas as pd
import numpy as np

In [2]:
#       - Importación -

df = pd.read_csv('nyc-rolling-sales.csv', dtype=str)
df = df.replace('-', '', regex=True)
df.columns = df.columns.str.replace(' ', '_')
df.shape

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1492\1933697466.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('-', '', regex=True)


(84548, 26)

In [3]:
#       - Formato -

convert_types = {
    'Unnamed:_0': 'str',
    'BOROUGH': 'int',
    'NEIGHBORHOOD': 'str',
    'BUILDING_CLASS_CATEGORY': 'str',
    'TAX_CLASS_AT_PRESENT': 'category',
    'BLOCK': 'int',
    'LOT': 'int',
    'EASE-MENT': 'str',
    'BUILDING_CLASS_AT_PRESENT': 'category',
    'ADDRESS': 'str',
    'APARTMENT_NUMBER': 'str',
    'ZIP_CODE': 'int',
    'RESIDENTIAL_UNITS': 'int',
    'COMMERCIAL_UNITS': 'int',
    'TOTAL_UNITS': 'int',
    'LAND_SQUARE_FEET': 'int',
    'GROSS_SQUARE_FEET': 'int',
    'YEAR_BUILT': 'int',
    'TAX_CLASS_AT_TIME_OF_SALE': 'category',
    'BUILDING_CLASS_AT_TIME_OF_SALE': 'category',
    'SALE_PRICE': 'float',
    'SALE_DATE': 'datetime64[ns]',
    'max_temp_celsius': 'float',
    'season': 'category',
}

for col, dtype in convert_types.items():
    if col not in df.columns:
        continue
    if dtype == 'datetime64[ns]':
        df[col] = pd.to_datetime(df[col], errors='coerce')
    elif dtype in ['int', 'float']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else:
        df[col] = df[col].astype(dtype)

print(df.columns)

Index(['Unnamed:_0', 'BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE', 'max_temp_celsius', 'min_temp_celsius',
       'avg_temp_celsius', 'season'],
      dtype='object')


In [4]:
#       - Filtro -
df['SALE_PRICE'] = pd.to_numeric(df['SALE_PRICE'], errors='coerce')
df.shape

(84548, 26)

In [5]:
#       - Limpieza -
df = df[df['SALE_PRICE'] > 10000]
df = df[~((df['RESIDENTIAL_UNITS'] > 3))]
df = df[~((df['COMMERCIAL_UNITS'] > 3))]
df_ready = df.drop(['Unnamed:_0','NEIGHBORHOOD','BUILDING_CLASS_CATEGORY','TAX_CLASS_AT_PRESENT',"BLOCK","LOT",'EASE-MENT',
                    'BUILDING_CLASS_AT_PRESENT', 'ADDRESS','APARTMENT_NUMBER','COMMERCIAL_UNITS','TOTAL_UNITS','LAND_SQUARE_FEET',
                    'min_temp_celsius','avg_temp_celsius'],axis='columns')

print(df_ready.columns)
df_ready.shape

Index(['BOROUGH', 'ZIP_CODE', 'RESIDENTIAL_UNITS', 'GROSS_SQUARE_FEET',
       'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_PRICE', 'SALE_DATE',
       'max_temp_celsius', 'season'],
      dtype='object')


(56008, 11)

In [6]:
#       - Depuración de GROSS_SQUARE_FEET -

df_ready['GROSS_SQUARE_FEET'] = df_ready['GROSS_SQUARE_FEET'].astype(str).str.replace(',', '').str.strip()
df_ready['GROSS_SQUARE_FEET'] = df_ready['GROSS_SQUARE_FEET'].replace(['', '-', '0'], np.nan)
df_ready['GROSS_SQUARE_FEET'] = pd.to_numeric(df_ready['GROSS_SQUARE_FEET'], errors='coerce')
df_ready['GROSS_SQUARE_FEET'] = df_ready['GROSS_SQUARE_FEET'].replace(0, np.nan)

#       - Imputación de GROSS_SQUARE_FEET -

missing_data = df_ready['GROSS_SQUARE_FEET'].isna()
imputation = df_ready.groupby('BUILDING_CLASS_AT_TIME_OF_SALE')['GROSS_SQUARE_FEET'].transform('median')
df_ready.loc[missing_data, 'GROSS_SQUARE_FEET'] = df_ready.loc[missing_data, 'GROSS_SQUARE_FEET'].fillna(imputation)
median_global = df_ready['GROSS_SQUARE_FEET'].median()

df_ready.loc[missing_data, 'GROSS_SQUARE_FEET'] = median_global

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1492\1805719715.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  imputation = df_ready.groupby('BUILDING_CLASS_AT_TIME_OF_SALE')['GROSS_SQUARE_FEET'].transform('median')


In [7]:
#       - Limpieza BUILDING_CLASS_AT_TIME_OF_SALE -

with open("eliminacion_clases.txt", "r") as archivo:
    lista = [linea.rstrip('\n') for linea in archivo]

df_ready = df_ready[~df_ready.BUILDING_CLASS_AT_TIME_OF_SALE.isin(lista)]
df_ready.shape

(52457, 11)

In [8]:
#       - Filtro GROSS_SQUARE_FEET, YEAR_BUILT, ZIP_CODE-

df_ready = df_ready[df_ready['GROSS_SQUARE_FEET'] > 500]
df_ready = df_ready[df_ready['GROSS_SQUARE_FEET'] < 4000]
df_ready = df_ready[((df_ready["TAX_CLASS_AT_TIME_OF_SALE"] == "1") | (df_ready["TAX_CLASS_AT_TIME_OF_SALE"] == "2"))]
df_ready = df_ready[df_ready["YEAR_BUILT"] > 0]
df_ready = df_ready[df_ready["ZIP_CODE"] > 0]
df_ready.shape

(48740, 11)

In [9]:
#       - Cambio de valores 0 a 1 RESIDENTIAL_UNITS-
df_ready = df_ready.replace({'RESIDENTIAL_UNITS': 0}, 1)
df_ready["RESIDENTIAL_UNITS"].value_counts()

RESIDENTIAL_UNITS
1    37309
2     9244
3     2187
Name: count, dtype: int64

In [10]:
#       - Eliminar valores con frecuencia <=10 YEAR_BUILT-

freq = df_ready['YEAR_BUILT'].value_counts()
ind = freq[freq <= 10].index
df_ready = df_ready[~df_ready['YEAR_BUILT'].isin(ind)]
df_ready.shape

(48678, 11)

In [11]:
#       - Exportación -
df_ready.to_csv('training_NYC.csv', index=False, encoding='utf-8', float_format='%.2f')